In [ ]:
from Crypto.Cipher import AES
import hashlib
import random


# /usr/share/dict/words from
# https://gist.githubusercontent.com/wchargin/8927565/raw/d9783627c731268fb2935a731a618aa8e95cf465/words
# with open("/usr/share/dict/words") as f:
#     words = [w.strip() for w in f.readlines()]
# keyword = random.choice(words)

# KEY = hashlib.md5(keyword.encode()).digest()
KEY = "Null"
FLAG = "Null"

# @chal.route('/passwords_as_keys/decrypt/<ciphertext>/<password_hash>/')
def decrypt(ciphertext, password_hash):
    ciphertext = bytes.fromhex(ciphertext)
    key = bytes.fromhex(password_hash)

    cipher = AES.new(key, AES.MODE_ECB)
    try:
        decrypted = cipher.decrypt(ciphertext)
    except ValueError as e:
        return {"error": str(e)}

    return {"plaintext": decrypted.hex()}


# @chal.route('/passwords_as_keys/encrypt_flag/')
def encrypt_flag():
    cipher = AES.new(KEY, AES.MODE_ECB)
    encrypted = cipher.encrypt(FLAG.encode())

    return {"ciphertext": encrypted.hex()}

In [8]:
from Crypto.Cipher import AES
import hashlib
import random
import requests

KEY = None
FLAG = None

def decrypt(ciphertext, password_hash):
    ciphertext = bytes.fromhex(ciphertext)
    key = bytes.fromhex(password_hash)

    cipher = AES.new(key, AES.MODE_ECB)
    try:
        decrypted = cipher.decrypt(ciphertext)
    except ValueError as e:
        return {"error": str(e)}

    return {"plaintext": decrypted.hex()}

request = requests.get("https://aes.cryptohack.org/passwords_as_keys/encrypt_flag/")
ciphertext = request.json()["ciphertext"]
print(f"ciphertext: {ciphertext}")

url_key = "https://gist.githubusercontent.com/wchargin/8927565/raw/d9783627c731268fb2935a731a618aa8e95cf465/words"
request = requests.get(url_key, timeout=10)
request.raise_for_status() 

words = [w.strip() for w in request.text.splitlines() if w.strip()]

sample = b"crypto".hex()
for key in words:
    # KEY = hashlib.md5(key.encode()).hexdigest()
    # url = f"https://aes.cryptohack.org/passwords_as_keys/decrypt/{ciphertext}/{KEY}/"
    # request = requests.get(url, timeout=10)
    # plaintext = request.json()["plaintext"]
    
    KEY = hashlib.md5(key.encode()).hexdigest()
    request = decrypt(ciphertext, KEY)
    
    plaintext = request["plaintext"]
    
    if plaintext.startswith(sample):
        print(bytes.fromhex(plaintext).decode('utf-8'), key)
        break
    

ciphertext: c92b7734070205bdf6c0087a751466ec13ae15e6f1bcdd3f3a535ec0f4bbae66
crypto{k3y5__r__n07__p455w0rdz?} bluebell
